<a href="https://colab.research.google.com/github/EgovisionPolito/aml23-ego/blob/master/colab_runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training different models

In [1]:
# Clone the public repository (or your repository)
# at the moment I am cloning a single branch
!git clone --branch reset --single-branch https://github.com/nicolo-vallania/aml23-ego.git aml23-ego

# Installing missing dependencies
!pip install omegaconf coloredlogs wandb

Cloning into 'aml23-ego'...
remote: Enumerating objects: 158, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 158 (delta 15), reused 22 (delta 3), pack-reused 107 (from 1)
Receiving objects: 100% (158/158), 1.17 GiB | 28.21 MiB/s, done.
Resolving deltas: 100% (38/38), done.
Updating files: 100% (67/67), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.1/309.1 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

## RGB

### GRU

In [36]:

%%bash

cd aml23-ego

num_frames_per_clip=(5 10 25)
dense_sampling=("True" "False")

layers=(1 2 3)
dropout=(0 0.33 0.5)

for frames in "${num_frames_per_clip[@]}"
do
  for n in "${layers[@]}"
  do
    for mode in "${dense_sampling[@]}"
    do
      for drop_prob in "${dropout[@]}"
      do
        if [ "$mode" = "True" ]
        then
          sampling="dense"
        else
          sampling="uniform"
        fi
        if [ "$drop_prob" = "0" ]
        then
          drop="low"
        elif [ "$drop_prob" = "0.33" ]
        then
          drop="medium"
        else
          drop="high"
        fi
        echo "Running: frames=$frames, sampling=$sampling, dropout=$drop_prob, layers=$n"
        python train_classifier.py name=GRU_RGB_frames_${frames}_sampling_${sampling}_dropout_${drop}_with_${n}_layers \
          config=configs/train_RGB.yaml \
          train.dense_sampling.RGB=$mode \
          train.num_frames_per_clip.RGB=$frames \
          dataset.shift=D1-D1\
          dataset.RGB.features_name="${sampling}_${frames}"\
          models.RGB.model=GRU_RGB \
          models.RGB.dropout=${drop_prob} \
          models.RGB.n_layers=$n

      done
    done
  done
done


Process is terminated.


### LSTM

In [26]:
%%bash

cd aml23-ego

num_frames_per_clip=(5 10 25)
dense_sampling=("True" "False")
layers=(1 2 3)
dropout=(0 0.33 0.5)

for frames in "${num_frames_per_clip[@]}"
do
  for n in "${layers[@]}"
  do
    for mode in "${dense_sampling[@]}"
    do
      for drop_prob in "${dropout[@]}"
      do
        if [ "$mode" = "True" ]
        then
          sampling="dense"
        else
          sampling="uniform"
        fi
        if [ "$drop_prob" = "0" ]
        then
          drop="low"
        elif [ "$drop_prob" = "0.33" ]
        then
          drop="medium"
        else
          drop="high"
        fi
        echo "Running: frames=$frames, sampling=$sampling, dropout=$drop_prob, layers=$n"
        python train_classifier.py \
          name=LSTM_RGB_frames_${frames}_sampling_${sampling}_dropout_${drop}_with_${n}_layers \
          config=configs/train_RGB.yaml \
          train.dense_sampling.RGB=$mode \
          train.num_frames_per_clip.RGB=$frames \
          dataset.RGB.features_name="${sampling}_${frames}"\
          dataset.shift=D1-D1 \
          models.RGB.model=LSTM_RGB \
          models.RGB.dropout=${drop_prob} \
          models.RGB.n_layers=$n
      done
    done
  done
done


Process is terminated.


### MLP

In [ ]:
%%bash

cd aml23-ego

num_frames_per_clip=(5 10 25)
dense_sampling=("True" "False")
dropout=(0 0.33 0.5)

for frames in "${num_frames_per_clip[@]}"
do
  for mode in "${dense_sampling[@]}"
  do
    for drop_prob in "${dropout[@]}"
    do
      if [ "$mode" = "True" ]
      then
        sampling="dense"
      else
        sampling="uniform"
      fi

      if [ "$drop_prob" = "0" ]
      then
        drop="low"
      elif [ "$drop_prob" = "0.33" ]
      then
        drop="medium"
      else
        drop="high"
      fi

      echo "Running: frames=$frames, sampling=$sampling, dropout=$drop_prob"
      python train_classifier.py \
        name=MLP_frames_${frames}_sampling_${sampling}_dropout_${drop} \
        config=configs/train_RGB.yaml \
        train.dense_sampling.RGB=$mode \
        train.num_frames_per_clip.RGB=$frames \
        dataset.shift=D1-D1 \
        models.RGB.model=MLP \
        models.RGB.n_layers=1\
        dataset.RGB.features_name="${sampling}_${frames}"\
        models.RGB.dropout=$drop_prob
    done
  done
done


### TSN (?)